In [1]:
#!pip install format_date

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import  Dense,Input,Bidirectional,Concatenate,Permute,Dot,LSTM,Multiply
from tensorflow.keras.layers import RepeatVector,Activation,Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model,Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline


from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
m = 1000
dataset,human_vocab,machine_vocab,inv_machine_vocab = load_dataset(m)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1507.76it/s]


In [4]:
fake = Faker()

# Generate random data
print(fake.name())         # Random name
print(fake.address())      # Random address
print(fake.email())        # Random email
print(fake.date_of_birth())  # Random date of birth
print(fake.text()) 

Melissa Schmidt
17677 Ronald Spur Apt. 575
Cardenashaven, CT 03712
jessica70@example.net
1942-01-13
Health system a specific.
Year position customer that fish game. Some use those able director.


In [5]:
dataset[:9]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('monday august 19 2024', '2024-08-19'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('22 may 1988', '1988-05-22'),
 ('8 jul 2008', '2008-07-08')]

In [6]:
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [7]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

In [8]:
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

In [9]:
tx = 30
ty = 10
X,Y,Xoh ,Yoh = preprocess_data(dataset,human_vocab,machine_vocab,tx,ty)

In [10]:
print('the shape of the X is :- ',X.shape)
print('the shape of the Y is :- ',Y.shape)
print('the shape of the Xoh is :- ',Xoh.shape)
print('the shape of the Yoh is :- ',Yoh.shape)


the shape of the X is :-  (1000, 30)
the shape of the Y is :-  (1000, 10)
the shape of the Xoh is :-  (1000, 30, 37)
the shape of the Yoh is :-  (1000, 10, 11)


In [11]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2 - Neural machine translation with attention

In [12]:
repeator = RepeatVector(tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1,activation='relu')
activator = Activation(softmax,name='attention_weights')
docer = Dot(axes = 1)

In [13]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = docer([alphas, a])
    ### END CODE HERE ###
    
    return context

In [14]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [15]:
output_layer

<Dense name=dense_1, built=False>

In [16]:
def model(tx,ty,n_a,n_s,human_vocab_size,machine_vocab_size):
    X = Input(shape=(tx,human_vocab_size))
    so = Input(shape=(n_s,),name='so')
    co = Input(shape=(n_s,),name='co')
    s = so
    c = co

    outputs = []
    a = Bidirectional(LSTM(n_a,return_sequences=True))(X)
    for t in range(ty):
        context = one_step_attention(a,s)
        s,_,c = post_activation_LSTM_cell(context, initial_state = [s, c])
        out = output_layer(s)
        outputs.append(out)
    model = Model(inputs=[X,so,co],outputs =outputs)
    return model

In [17]:
model = model(tx, ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [18]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 37)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ so (InputLayer)               │ (None, 128)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 30, 128)           │          52,224 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ repeat_vector (RepeatVector)  │ (None, 30, 128)           │               0 │ so[0][0], lstm[0][0],      │
│                               │                           │                 │ lstm[1][0], lstm[2][0],    │
│                               │                           │                 │ lstm[3][0], lstm[4][0],    │
│                               │                           │                 │ lstm[5][0], lstm[6][0],    │
│                               │                           │                 │ lstm[7][0], lstm[8][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 30, 256)           │               0 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[0][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[1][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[2][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[3][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[4][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[5][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[6][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[7][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[8][0],       │
│                               │                           │                 │ bidirectional[0][0],       │
│                               │                           │                 │ repeat_vector[9][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 30, 1)             │             25

 Total params: 185,484 (724.55 KB)

 Trainable params: 185,484 (724.55 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
out = model.compile(optimizer=Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01),
                    metrics=['accuracy']*10,
                    loss='categorical_crossentropy')

C:\Users\Adams\AppData\Roaming\Python\Python312\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [20]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [21]:
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 52s 139ms/step - dense_1_accuracy: 0.0000e+00 - dense_1_accuracy_1: 0.1855 - dense_1_accuracy_2: 0.0727 - dense_1_accuracy_3: 0.0375 - dense_1_accuracy_4: 0.6125 - dense_1_accuracy_5: 0.2671 - dense_1_accuracy_6: 0.0357 - dense_1_accuracy_7: 0.5945 - dense_1_accuracy_8: 0.1202 - dense_1_accuracy_9: 0.0498 - dense_1_loss: 2.7173 - loss: 22.6190
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - dense_1_accuracy: 0.0000e+00 - dense_1_accuracy_1: 0.5017 - dense_1_accuracy_2: 0.1788 - dense_1_accuracy_3: 0.0804 - dense_1_accuracy_4: 0.9691 - dense_1_accuracy_5: 5.6566e-04 - dense_1_accuracy_6: 0.0000e+00 - dense_1_accuracy_7: 0.9805 - dense_1_accuracy_8: 0.0649 - dense_1_accuracy_9: 0.0467 - dense_1_loss: 2.6989 - loss: 20.5122
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 132ms/step - dense_1_accuracy: 0.0401 - dense_1_accuracy_1: 0.6052 - dense_1_accuracy_2: 0.2669 - dense_1_accuracy_3: 0.0690 - dense_1_accuracy_4: 0.9751 - dense_1_accuracy_5: 0.0148 -

In [22]:
#model.load_weights('mode.h5')

In [23]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 37, 1000, 1000
